In [24]:
import warnings
warnings.simplefilter(action='ignore')

from datetime import datetime
from meteostat import Point, Hourly, Stations
import pandas as pd
from tqdm import tqdm



# Define time range 
start = datetime(2006, 1, 1)
end = datetime(2025, 1, 1)

# List of some U.S. regions with coordinates
regions = [
    # central and river plains
    ("Tennessee", 35.6000, -88.8000),
    ("Kentucky", 37.8393, -84.2700),
    ("Mississippi", 32.3547, -89.3985),
    ("Louisiana", 31.2000, -92.4000),
    ("New Mexico", 34.5199, -105.8701),
    ("Iowa", 41.6000, -93.6000),
    ("Missouri", 38.5739, -92.6038),
    ("Illinois", 39.8000, -89.6000),
    ("Indiana", 39.8000, -86.1000),
    ("Ohio", 40.0000, -83.0000),
    ("Arkansas", 34.7465, -92.2896),
    
    ("North Dakota", 46.8000, -100.8000),
    ("South Dakota", 44.2998, -99.4388),
    ("Nebraska", 41.5000, -99.7000),
    ("Kansas", 38.5000, -98.0000),
    ("Oklahoma", 35.5000, -97.5000),
    ("Texas", 35.2000, -101.8000),
    ("Montana", 47.1000, -104.7000),
    ("Wyoming", 42.1000, -104.2000),
    ("Colorado", 39.5501, -105.7821),
    
    # east coast
    ("Maine", 44.3106, -69.7795),
    ("New Hampshire", 43.2081, -71.5376),
    ("Massachusetts", 42.3601, -71.0589),
    ("Rhode Island", 41.8240, -71.4128),
    ("Connecticut", 41.7658, -72.6734),
    ("New York", 40.7128, -74.0060),
    ("New Jersey", 40.2206, -74.7699),
    ("Maryland", 38.9784, -76.4922),
    ("North Carolina", 35.7796, -78.6382),
    ("South Carolina", 34.0007, -81.0348),
    ("Georgia", 33.7490, -84.3880),
    ("Florida", 30.4383, -84.2807),
    ("Delaware", 39.0000, -75.5000),
    ("Virginia", 37.5000, -78.7500)
]

In [18]:
# Collect data
all_data = []
for name, lat, lon in tqdm(regions, desc="Fetching weather"):
    stations = Stations().nearby(lat, lon).fetch(10)
    for station_id, row in stations.iterrows():
        if row['hourly_start'] <= start and row['hourly_end'] >= end:
            df = Hourly(station_id, start, end).fetch()
            break
    df.reset_index(inplace=True)
    df['region'] = name
    all_data.append(df[['time', 'region', 'temp']])

# Combine and save
df_all = pd.concat(all_data, ignore_index=True)


Fetching weather: 100%|██████████| 34/34 [00:03<00:00, 11.25it/s]


In [19]:
df_pivot = df_all.pivot(index='time', columns='region', values='temp')
df_pivot.columns = [col.lower().replace(" ", "_") + "_temp" for col in df_pivot.columns]
df_pivot.reset_index(inplace=True)
df_pivot.to_csv("us_eastern_regions_hourly_temperature_2006_to_2025.csv", index=False)

In [25]:
df_pivot.head()

,time,arkansas_temp,colorado_temp,connecticut_temp,delaware_temp,florida_temp,georgia_temp,illinois_temp,indiana_temp,iowa_temp,...,north_dakota_temp,ohio_temp,oklahoma_temp,rhode_island_temp,south_carolina_temp,south_dakota_temp,tennessee_temp,texas_temp,virginia_temp,wyoming_temp
0,2006-01-01 00:00:00,NaN,NaN,NaN,NaN,19.4,NaN,1.1,1.7,1.1,...,-2.8,NaN,NaN,-0.6,NaN,NaN,NaN,9.4,6.0,NaN
1,2006-01-01 01:00:00,NaN,-1.0,-2.8,6.1,19.4,6.7,-0.6,1.7,1.1,...,-2.8,1.7,11.7,-0.6,14.4,-1.1,8.9,10.0,7.0,11.7
2,2006-01-01 02:00:00,NaN,-3.0,-2.2,5.6,17.8,3.9,-0.6,1.7,0.6,...,-2.8,1.7,13.3,0.0,12.8,-1.1,8.3,8.3,6.0,8.3
3,2006-01-01 03:00:00,NaN,-4.0,-2.2,5.0,16.1,3.3,0.6,1.7,0.0,...,-3.3,1.7,13.3,0.0,11.1,-1.7,3.3,8.3,3.0,8.3
4,2006-01-01 04:00:00,NaN,-4.0,-1.7,3.9,15.0,2.8,0.0,0.6,0.0,...,-3.3,1.7,13.9,0.0,8.9,-2.2,5.6,7.2,3.0,7.2
